In [2]:
#!pip install dlib

  Using cached dlib-19.24.4.tar.gz (3.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Failed to build dlib


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\AJCag\anaconda3\python.exe' 'C:\Users\AJCag\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' build_wheel 'C:\Users\AJCag\AppData\Local\Temp\tmpmsf7w_e6'
       cwd: C:\Users\AJCag\AppData\Local\Temp\pip-install-62hcn41w\dlib_6563e946da8345df8ca1da74c7609c49
  Complete output (73 lines):
  running bdist_wheel
  running build
  running build_ext
  <string>:125: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  Building extension for Python 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
  Invoking CMake setup: 'cmake C:\Users\AJCag\AppData\Local\Temp\pip-install-62hcn41w\dlib_6563e946da8345df8ca1da74c7609c49\tools\python -DCMAKE_LIBRARY_OUTPUT_DIRECTORY=C:\Users\AJCag\AppData\Local\Temp\pip-install-62hcn41w\dlib_6563e946da8345df8ca1da74c7609c49\build\lib.win-amd64-cpython-39 -DPYTHON_EXECUTABLE=C:\Users\AJCag\anaconda3\python.exe -D

In [1]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
from sklearn import preprocessing
import dlib
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import timeit
import pickle
import random
import sklearn_crfsuite
from sklearn_crfsuite import metrics

ModuleNotFoundError: No module named 'dlib'

In [ ]:
# Used to read OCR dataset
def l2i(a):
    return int(ord(a) - ord('a'))

def i2l(i):
    if i >= 0:
        return chr(i + ord('a'))
    else:
        return '_'

def iors(s):
    try:
        return int(s)
    except ValueError:
        return s

In [ ]:
def read_OCR(filename, n_features):
    with open(filename) as F:
        dataset = {
            'ids': [],
            'labels': [],
            'labelDic': {},
            'next_ids': [],
            'word_ids': [],
            'positions': [],
            'folds': [],
            'features': []
        }
        for str_line in F.readlines():
            line0 = list(map(iors, filter(None, re.split('\t', str_line.strip()))))
            dataset['ids'].append(int(line0.pop(0)))
            dataset['labels'].append(l2i(line0.pop(0)))
            if dataset['labels'][-1] in dataset['labelDic']:
                dataset['labelDic'][dataset['labels'][-1]] += 1
            else:
                dataset['labelDic'][dataset['labels'][-1]] = 1
            dataset['next_ids'].append(int(line0.pop(0)))
            dataset['word_ids'].append(int(line0.pop(0)))
            dataset['positions'].append(int(line0.pop(0)))
            dataset['folds'].append(int(line0.pop(0)))
            dataset['features'].append(line0)
    return dataset

In [ ]:
# Load the OCR dataset
d = 128
dataset1 = read_OCR('OCRdataset/letter.data', d)

In [ ]:
# Illustrates OCR raw data
print("max of labels=", max(dataset1['labels']), " min of labels=", min(dataset1['labels']), 'num of labels=', len(dataset1['labelDic']))
print("labelDic.keys()=", list(map(i2l, dataset1['labelDic'].keys())))
print("Total number of lines=", len(dataset1['ids']))
print("The shape of features:", np.array(dataset1['features']).shape)

print("The first 10 ids:", dataset1['ids'][:10])
print("ids[0]=", dataset1['ids'][0])
print("labels[0]=", dataset1['labels'][0])
print("The 1st letter is ", i2l(dataset1['labels'][0]))
print("next_ids[0]=", dataset1['next_ids'][0])

In [ ]:
# Show the matrix into an image
def showFeatures(features, num):
    plt.figure(figsize=(num, 6))
    for i in range(num):
        npfeature = np.array(features[i])
        plt.subplot(1, num, i + 1)
        imshow(npfeature.reshape(16, 8), cmap='gray')
        plt.title(i)

showFeatures(dataset1['features'], 20)

In [ ]:
def structurize(dataset, N, L):
    d_features = len(dataset['features'][0])
    y = dataset['labels']
    X = dataset['features']
    next_id = dataset['next_ids']

    labels = np.zeros((N, L))
    features = np.zeros((N, L * d_features))

    # Extract only one structured example
    def extract(iN, loc):
        labels[iN] = y[loc:loc + L]
        features[iN] = np.array(X[loc:loc + L]).ravel().tolist()
        iN += 1
        return iN

    iN = 0
    iN = extract(iN, 0)

    for key, value in enumerate(y):
        if next_id[key] == -1:
            iN = extract(iN, key + 1)
            if iN == N:
                break

    c = list(zip(labels, features))
    random.shuffle(c)
    labels, features = zip(*c)

    return np.array(labels), np.array(features)

In [ ]:
class ThreeClassProblem:
    C = 1

    def __init__(self, samples, labels, L, K, d):
        self.L = L
        self.K = K
        self.d = d
        self.num_samples = len(samples)
        self.num_dimensions = (L * K * d + 1) + (L - 1)
        self.samples = samples
        self.labels = labels
        self.loss_for_loop = True

    def make_psi(self, x, label):
        psi = dlib.vector()
        psi.resize(self.num_dimensions)
        psi[0] = 1  # The bias term

        for i in range(self.L):
            for j in range(self.K):
                for k in range(self.d):
                    psi[i * self.K * self.d + j * self.d + k] = x[i * self.d + k] * (1 if label[i] == j else 0)
        for i in range(self.L - 1):
            psi[self.L * self.K * self.d + 1 + i] = 1 if label[i] != label[i + 1] else 0

        return psi

    def get_truth_joint_feature_vector(self, idx):
        return self.make_psi(self.samples[idx], self.labels[idx])

    def separation_oracle(self, idx, current_solution):
        samp = self.samples[idx]
        psi = [0] * self.num_dimensions
        max1 = -1e10
        max_scoring_label = [0] * self.L  # Initialize max_scoring_label for ICM search
        for k in range(Niter):
            for iL in range(self.L):  # Iterate over the window length
                for i in range(self.K):  # Change different label for the search of a structured label
                    tmp_label = list(max_scoring_label)  # New a list to avoid modifying the max_scoring_label
                    tmp_label[iL] = i  # Take turns to modify the structured label from left to right. The guessed structured label.
                    tmp_psi = self.make_psi(samp, tmp_label)  # Make a new Psi for the guessed structured label
                    score1 = dlib.dot(current_solution, tmp_psi)

                    loss1 = sum([1.0 if self.labels[idx][j] != tmp_label[j] else 0.0 for j in range(self.L)])

                    if max1 < score1 + loss1:  # Search for the maximum and update loss, max_scoring_label, and psi
                        max1 = score1 + loss1
                        loss = loss1
                        max_scoring_label[iL] = i
                        psi = tmp_psi

        return loss, psi

In [ ]:
class StructuredSVM:
    def __init__(self, C, L, K, d, Niter):
        self.C = C
        self.L = L
        self.K = K
        self.d = d
        self.Niter = Niter

    def fit(self, samples, labels):
        self.samples = samples
        self.labels = labels
        problem = ThreeClassProblem(samples, labels, self.L, self.K, self.d)
        start_train = timeit.default_timer()
        self.weights = dlib.solve_structural_svm_problem(problem)
        end_train = timeit.default_timer()
        self.train_time = end_train - start_train
        print("Training time elapsed:", self.train_time, "s")
        pickle.dump(self.weights, open('weights_structured_svm.obj', 'wb'))

    def predict(self, samples):
        predictions = []
        problem = ThreeClassProblem(samples, [], self.L, self.K, self.d)
        for idx, samp in enumerate(samples):
            max1 = -1e10
            max_scoring_label = [0] * self.L  # Initialize max_scoring_label for ICM search
            for k in range(self.Niter):
                for iL in range(self.L):  # Iterate over the window length
                    for i in range(self.K):  # Change different label for the search of a structured label
                        tmp_label = list(max_scoring_label)  # New a list to avoid modifying the max_scoring_label
                        tmp_label[iL] = i  # Take turns to modify the structured label from left to right
                        tmp_psi = problem.make_psi(samp, tmp_label)  # Make a new Psi for the guessed structured label
                        score1 = dlib.dot(self.weights, tmp_psi)
                        if max1 < score1:
                            max1 = score1
                            max_scoring_label[iL] = i
            predictions.append(max_scoring_label)
        return predictions

    def accuracy(self, samples, labels):
        predictions = self.predict(samples)
        errCnt = sum([1 if predictions[i] != labels[i] else 0 for i in range(len(predictions))])
        return 1.0 - float(errCnt) / float(len(predictions))

In [ ]:

# Function to run structured SVM experiments
def run_svm_experiment(dataset, window_size, N, d, train_size, test_size):
    labels, features = structurize(dataset, N, window_size)
    le = preprocessing.LabelEncoder()
    nplabels = le.fit_transform(labels.ravel()).reshape(labels.shape)
    npsamples = np.hstack([np.ones((N, 1)), features])
    K = len(le.classes_)

    tr_labels = nplabels[:train_size].astype(int).tolist()
    tr_samples = npsamples[:train_size].astype(int).tolist()
    te_labels = nplabels[train_size:train_size + test_size].astype(int).tolist()
    te_samples = npsamples[train_size:train_size + test_size].astype(int).tolist()

    model = StructuredSVM(C, window_size, K, d, Niter)
    model.fit(tr_samples, tr_labels)
    train_time = model.train_time
    train_acc = model.accuracy(tr_samples, tr_labels)
    start_test = timeit.default_timer()
    test_acc = model.accuracy(te_samples, te_labels)
    test_time = timeit.default_timer() - start_test

    return {
        "method": "SVM",
        "window_size": window_size,
        "train_size": train_size,
        "test_size": test_size,
        "train_accuracy": train_acc,
        "test_accuracy": test_acc,
        "train_time": train_time,
        "test_time": test_time
    }

In [ ]:
class MaximumMarginMarkovNetworks:
    def __init__(self, C, L, K, d, Niter):
        self.C = C
        self.L = L
        self.K = K
        self.d = d
        self.Niter = Niter

    def fit(self, samples, labels):
        self.samples = samples
        self.labels = labels
        problem = ThreeClassProblem(samples, labels, self.L, self.K, self.d)
        start_train = timeit.default_timer()
        self.weights = dlib.solve_structural_svm_problem(problem)
        end_train = timeit.default_timer()
        self.train_time = end_train - start_train
        print("Training time elapsed:", self.train_time, "s")
        pickle.dump(self.weights, open('weights_m3n.obj', 'wb'))

    def predict(self, samples):
        predictions = []
        problem = ThreeClassProblem(samples, [], self.L, self.K, self.d)
        for idx, samp in enumerate(samples):
            max1 = -1e10
            max_scoring_label = [0] * self.L  # Initialize max_scoring_label for ICM search
            for k in range(self.Niter):
                for iL in range(self.L):  # Iterate over the window length
                    for i in range(self.K):  # Change different label for the search of a structured label
                        tmp_label = list(max_scoring_label)  # New a list to avoid modifying the max_scoring_label
                        tmp_label[iL] = i  # Take turns to modify the structured label from left to right
                        tmp_psi = problem.make_psi(samp, tmp_label)  # Make a new Psi for the guessed structured label
                        score1 = dlib.dot(self.weights, tmp_psi)
                        if max1 < score1:
                            max1 = score1
                            max_scoring_label[iL] = i
            predictions.append(max_scoring_label)
        return predictions

    def accuracy(self, samples, labels):
        predictions = self.predict(samples)
        errCnt = sum([1 if predictions[i] != labels[i] else 0 for i in range(len(predictions))])
        return 1.0 - float(errCnt) / float(len(predictions))

In [ ]:
# Function to run M3N experiments
def run_mmmn_experiment(dataset, window_size, N, d, train_size, test_size):
    labels, features = structurize(dataset, N, window_size)
    le = preprocessing.LabelEncoder()
    nplabels = le.fit_transform(labels.ravel()).reshape(labels.shape)
    npsamples = np.hstack([np.ones((N, 1)), features])
    K = len(le.classes_)

    tr_labels = nplabels[:train_size].astype(int).tolist()
    tr_samples = npsamples[:train_size].astype(int).tolist()
    te_labels = nplabels[train_size:train_size + test_size].astype(int).tolist()
    te_samples = npsamples[train_size:train_size + test_size].astype(int).tolist()

    model = MaximumMarginMarkovNetworks(C, window_size, K, d, Niter)
    model.fit(tr_samples, tr_labels)
    train_time = model.train_time
    train_acc = model.accuracy(tr_samples, tr_labels)
    start_test = timeit.default_timer()
    test_acc = model.accuracy(te_samples, te_labels)
    test_time = timeit.default_timer() - start_test

    return {
        "method": "M3N",
        "window_size": window_size,
        "train_size": train_size,
        "test_size": test_size,
        "train_accuracy": train_acc,
        "test_accuracy": test_acc,
        "train_time": train_time,
        "test_time": test_time
    }

In [ ]:
def prepare_data_for_crf(samples, labels):
    data = []
    for sample, label in zip(samples, labels):
        features = []
        for i in range(len(label)):
            feature = {'bias': 1.0}
            for j in range(d):
                feature[f'x{i}_{j}'] = sample[i * d + j]
            features.append((feature, str(label[i])))
        data.append(features)
    return data

In [ ]:
# Function to run CRF experiments
def run_crf_experiment(dataset, window_size, N, d, train_size, test_size):
    labels, features = structurize(dataset, N, window_size)
    le = preprocessing.LabelEncoder()
    nplabels = le.fit_transform(labels.ravel()).reshape(labels.shape)
    npsamples = np.hstack([np.ones((N, 1)), features])
    K = len(le.classes_)

    tr_labels = nplabels[:train_size].astype(int).tolist()
    tr_samples = npsamples[:train_size].astype(int).tolist()
    te_labels = nplabels[train_size:train_size + test_size].astype(int).tolist()
    te_samples = npsamples[train_size:train_size + test_size].astype(int).tolist()

    tr_data = prepare_data_for_crf(tr_samples, tr_labels)
    te_data = prepare_data_for_crf(te_samples, te_labels)
    X_train = [x for x, y in tr_data]
    y_train = [y for x, y in tr_data]
    X_test = [x for x, y in te_data]
    y_test = [y for x, y in te_data]

    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=False
    )
    start_train = timeit.default_timer()
    crf.fit(X_train, y_train)
    train_time = timeit.default_timer() - start_train
    y_pred_train = crf.predict(X_train)
    train_acc = metrics.flat_accuracy_score(y_train, y_pred_train)
    start_test = timeit.default_timer()
    y_pred = crf.predict(X_test)
    test_time = timeit.default_timer() - start_test
    test_acc = metrics.flat_accuracy_score(y_test, y_pred)

    return {
        "method": "CRF",
        "window_size": window_size,
        "train_size": train_size,
        "test_size": test_size,
        "train_accuracy": train_acc,
        "test_accuracy": test_acc,
        "train_time": train_time,
        "test_time": test_time
    }

In [ ]:
# Define constants
N = 5000  # Number of examples
d = 128  # Length of a feature
Niter = 2  # Number of iterations for ICM search
C = 1  # Regularization parameter

In [ ]:
# Splits
splits = [(1000, 4000), (2500, 2500), (4000, 1000)]

In [ ]:
# Test different window sizes
window_sizes = [2, 3, 5]

In [ ]:
svm_train_accuracies = []
svm_test_accuracies = []
svm_train_times = []
svm_test_times = []

m3n_train_accuracies = []
m3n_test_accuracies = []
m3n_train_times = []
m3n_test_times = []

crf_train_accuracies = []
crf_test_accuracies = []
crf_train_times = []
crf_test_times = []

In [ ]:
for train_size, test_size in splits:
    for window_size in window_sizes:
        labels1, features1 = structurize(dataset1, N, window_size)
        le1 = preprocessing.LabelEncoder()
        nplabels1 = le1.fit_transform(labels1.ravel()).reshape(labels1.shape)
        npsamples1 = np.hstack([np.ones((N, 1)), features1])
        K1 = len(le1.classes_)

        tr_labels = nplabels1[:train_size].astype(int).tolist()
        tr_samples = npsamples1[:train_size].astype(int).tolist()
        te_labels = nplabels1[train_size:train_size + test_size].astype(int).tolist()
        te_samples = npsamples1[train_size:train_size + test_size].astype(int).tolist()

        # Train and evaluate Structured SVM
        structured_svm = StructuredSVM(C, window_size, K1, d, Niter)
        structured_svm.fit(tr_samples, tr_labels)
        svm_train_times.append(structured_svm.train_time)
        svm_train_acc = structured_svm.accuracy(tr_samples, tr_labels)
        svm_train_accuracies.append(svm_train_acc)

        start_test = timeit.default_timer()
        svm_test_acc = structured_svm.accuracy(te_samples, te_labels)
        end_test = timeit.default_timer()
        svm_test_times.append(end_test - start_test)
        svm_test_accuracies.append(svm_test_acc)
        print(f"Training accuracy (SVM, window_size={window_size}, train_size={train_size})=", svm_train_acc)
        print(f"Test accuracy (SVM, window_size={window_size}, test_size={test_size})=", svm_test_acc)

        # Train and evaluate M3N
        m3n = MaximumMarginMarkovNetworks(C, window_size, K1, d, Niter)
        m3n.fit(tr_samples, tr_labels)
        m3n_train_times.append(m3n.train_time)
        m3n_train_acc = m3n.accuracy(tr_samples, tr_labels)
        m3n_train_accuracies.append(m3n_train_acc)

        start_test = timeit.default_timer()
        m3n_test_acc = m3n.accuracy(te_samples, te_labels)
        end_test = timeit.default_timer()
        m3n_test_times.append(end_test - start_test)
        m3n_test_accuracies.append(m3n_test_acc)
        print(f"Training accuracy (M3N, window_size={window_size}, train_size={train_size})=", m3n_train_acc)
        print(f"Test accuracy (M3N, window_size={window_size}, test_size={test_size})=", m3n_test_acc)

        # Prepare data for CRF
        tr_data = prepare_data_for_crf(tr_samples, tr_labels)
        te_data = prepare_data_for_crf(te_samples, te_labels)
        X_train = [x for x, y in tr_data]
        y_train = [y for x, y in tr_data]
        X_test = [x for x, y in te_data]
        y_test = [y for x, y in te_data]

        # Train CRF model
        crf = sklearn_crfsuite.CRF(
            algorithm='lbfgs',
            c1=0.1,
            c2=0.1,
            max_iterations=100,
            all_possible_transitions=False
        )
        start_train = timeit.default_timer()
        crf.fit(X_train, y_train)
        end_train = timeit.default_timer()
        crf_train_times.append(end_train - start_train)
        y_pred_train = crf.predict(X_train)
        crf_train_acc = metrics.flat_accuracy_score(y_train, y_pred_train)
        crf_train_accuracies.append(crf_train_acc)

        start_test = timeit.default_timer()
        y_pred = crf.predict(X_test)
        end_test = timeit.default_timer()
        crf_test_times.append(end_test - start_test)
        crf_test_acc = metrics.flat_accuracy_score(y_test, y_pred)
        crf_test_accuracies.append(crf_test_acc)
        print(f"Training accuracy (CRF, window_size={window_size}, train_size={train_size})=", crf_train_acc)
        print(f"Test accuracy (CRF, window_size={window_size}, test_size={test_size})=", crf_test_acc)

In [ ]:
# Visualize the results
plt.figure(figsize=(18, 12))

# Plot for different splits
split_labels = ["1000/4000", "2500/2500", "4000/1000"]

for i, (train_size, test_size) in enumerate(splits):
    plt.subplot(3, 2, 2 * i + 1)
    plt.plot(window_sizes, svm_train_accuracies[i * len(window_sizes):(i + 1) * len(window_sizes)], label='SVM Train Accuracy')
    plt.plot(window_sizes, m3n_train_accuracies[i * len(window_sizes):(i + 1) * len(window_sizes)], label='M3N Train Accuracy')
    plt.plot(window_sizes, crf_train_accuracies[i * len(window_sizes):(i + 1) * len(window_sizes)], label='CRF Train Accuracy')
    plt.xlabel('Window Size')
    plt.ylabel('Training Accuracy')
    plt.title(f'Training Accuracy Comparison (Split: {split_labels[i]})')
    plt.legend()

    plt.subplot(3, 2, 2 * i + 2)
    plt.plot(window_sizes, svm_test_accuracies[i * len(window_sizes):(i + 1) * len(window_sizes)], label='SVM Test Accuracy')
    plt.plot(window_sizes, m3n_test_accuracies[i * len(window_sizes):(i + 1) * len(window_sizes)], label='M3N Test Accuracy')
    plt.plot(window_sizes, crf_test_accuracies[i * len(window_sizes):(i + 1) * len(window_sizes)], label='CRF Test Accuracy')
    plt.xlabel('Window Size')
    plt.ylabel('Testing Accuracy')
    plt.title(f'Testing Accuracy Comparison (Split: {split_labels[i]})')
    plt.legend()

plt.tight_layout()
plt.show()